In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['embeddings', 'train.csv', 'sample_submission.csv', 'test.csv']


In [8]:
import os
import time
import tensorflow as tf
import numpy as np # linear algebra
import random
import os 
os.environ['PYTHONHASHSEED'] = '11'
np.random.seed(22)
random.seed(33)
tf.set_random_seed(44)

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics
import gc

from keras import initializers, regularizers, constraints
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, Embedding, CuDNNGRU, LSTM, CuDNNLSTM
from keras.layers import Bidirectional, GlobalMaxPooling1D, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.initializers import glorot_uniform
from keras.callbacks import Callback
from keras.models import clone_model
import keras.backend as K

In [9]:
train_df = pd.read_csv("../input/train.csv")
##test_df = pd.read_csv("../input/test.csv")
print("Train shape : ",train_df.shape)
##print("Test shape : ",test_df.shape)

Train shape :  (1306122, 3)


In [22]:
del train_df

Extratcing the stopwords in English language

In [10]:
from nltk import corpus
stopwords=corpus.stopwords.words('english')

# fill up the missing values
train_X = train_df["question_text"].fillna("_na_").values
#test_X = test_df["question_text"].fillna("_na_").values


In [23]:
del stopwords

In [11]:
max_features = 90000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 50

Stemming (result includes words not in stopword list)

In [12]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

train_X = train_df['question_text']
train_X = train_X.tolist()

qid_train = train_df['qid']
qid_train = qid_train.tolist()

##test_X = test_df['question_text']
##test_X = test_X.tolist()


stemming_result_train = {}
ps = PorterStemmer()
for i in range(0,len(qid_train)):
    words = word_tokenize(train_X[i])
    stemming_result_train[qid_train[i]] = []
    for w in words:
        if w not in stopwords:
            stemming_result_train[qid_train[i]].append(ps.stem(w))
            

##stemming_result_test = []
##for i in range(len(test_X)):
    ##for w in test_X[i]:
        ##stemming_result_test.append(ps.stem(w))
    
# stemming_result_test

In [24]:
del qid_train, stemming_result_train

Other Preprocesing : converting into lower case, extracting only alphabetical characters

In [13]:
import re

stemmed_content_train = {}
lowered_content_train = {}
alphabet_content_train = {}
for qid, words in stemming_result_train.items():
    stemmed_content_train[qid] = " ".join(str(word) for word in words)
    lowered_content_train[qid] = stemmed_content_train[qid].lower()
    alphabet_content_train[qid] = re.sub(r'[^a-zA-Z ]',"", lowered_content_train[qid])
      
    
##stemmed_content_test = []
##lowered_content_test = []
##alphabet_content_test = []
##for i in range(len(stemming_result_test)):
    ##stemmed_content_test.append(" ".join(str(word) for word in stemming_result_test[i]))
    ##lowered_content_test.append(stemmed_content_test[i].lower())
    #alphabet_content[qid] = lowered_content[qid].replace('[^a-z ]','')
    ##alphabet_content_test.append(re.sub(r'[^a-zA-Z ]',"", lowered_content_test[i]))
    
preprocessed_content1_train = []
##preprocessed_content1_test = []

for qid, text in alphabet_content_train.items():
    preprocessed_content1_train.append(alphabet_content_train[qid])

##for qid, text in alphabet_content_test.items():
##   preprocessed_content1_test.append(alphabet_content_test[qid])

In [25]:
del stemmed_content_train,lowered_content_train,alphabet_content_train, preprocessed_content1_train

Other preprocessing : Tokenization, Padding/Truncating questions

In [14]:
# ## split to train and val
# train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=2018)

# Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features,
                     filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'’“”')


tokenizer.fit_on_texts(preprocessed_content1_train)
train_X = tokenizer.texts_to_sequences(preprocessed_content1_train)

# Pad the sentences 
trunc = 'pre'
train_X = pad_sequences(train_X, maxlen=maxlen, truncating=trunc)

##test_X = pad_sequences(test_X, maxlen=maxlen, truncating=trunc)

# Get the target values
train_y = train_df['target'].values

In [26]:
del tokenizer

In [27]:
gc.collect()

7

Splitting data into Train and Test

In [15]:
test_X = train_X[1000000:]
train_X = train_X[:1000000]
test_y = train_y[1000000:]
train_y = train_y[:1000000]

Creating Glove Embedding

In [16]:
EMBEDDING_FILE = '../input/embeddings/glove.840B.300d/glove.840B.300d.txt'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix_1 = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix_1[i] = embedding_vector

del embeddings_index; gc.collect() 

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  """


0

Creating Wiki News Embedding

In [17]:
EMBEDDING_FILE = '../input/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if len(o)>100)

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix_2 = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix_2[i] = embedding_vector
        
del embeddings_index; gc.collect() 

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  """


0

Creating Paragram Embedding

In [18]:
EMBEDDING_FILE = '../input/embeddings/paragram_300_sl999/paragram_300_sl999.txt'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding="utf8", errors='ignore') if len(o)>100)

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix_3 = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix_3[i] = embedding_vector

del embeddings_index; gc.collect() 

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  """


0

Creating Gensim Embedding

In [19]:
# https://www.kaggle.com/strideradu/word2vec-and-gensim-go-go-go
from gensim.models import KeyedVectors

EMBEDDING_FILE = '../input/embeddings/GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin'
embeddings_index = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix_4 = (np.random.rand(nb_words, embed_size) - 0.5) / 5.0
for word, i in word_index.items():
    if i >= max_features: continue
    if word in embeddings_index:
        embedding_vector = embeddings_index.get_vector(word)
        embedding_matrix_4[i] = embedding_vector
        
del embeddings_index; gc.collect()

0

In [31]:
del EMBEDDING_FILE

In [32]:
gc.collect()

69

Concatinating the embeddings

In [20]:
embedding_matrix = np.concatenate((embedding_matrix_1, embedding_matrix_2, embedding_matrix_3, embedding_matrix_4), axis=1)

del embedding_matrix_1, embedding_matrix_2, embedding_matrix_3, embedding_matrix_4
gc.collect()
np.shape(embedding_matrix)

(90000, 1200)

Utilities for Neural Network model

In [21]:
class ExponentialMovingAverage(Callback):
    def __init__(self, model, decay=0.999, mode='epoch', n=100):
        """
        mode: 'epoch': Do update_weights every epoch.
              'batch':                   every n batches.
        n   :
        """
        self.decay = decay
        self.mode = mode
        self.ema_model = clone_model(model)
        self.ema_model.set_weights(model.get_weights())
        self.n = n
        if self.mode is 'batch':
            self.cnt = 0
        self.ema_weights = [K.get_value(w) for w in model.trainable_weights]
        self.n_weights = len(self.ema_weights)
        super(ExponentialMovingAverage, self).__init__()

    def on_batch_end(self, batch, logs={}):
        if self.mode is 'batch':
            self.cnt += 1
            if self.cnt % self.n == 0:
                self.update_weights()

    def on_epoch_end(self, epoch, logs={}):
        if self.mode is 'epoch':
            self.update_weights()
        for var, w in zip(self.ema_model.trainable_weights, self.ema_weights):
            K.set_value(var, w)

    def update_weights(self):
        for w_old, var_new in zip(self.ema_weights, self.model.trainable_weights):
            w_old += (1 - self.decay) * (K.get_value(var_new) - w_old)

Attention Function

In [47]:
def _time_distributed_dense(x, w, b=None, dropout=None,
                            input_dim=None, output_dim=None,
                            timesteps=None, training=None):
    """Apply `y . w + b` for every temporal slice y of x.
    # Arguments
        x: input tensor.
        w: weight matrix.
        b: optional bias vector.
        dropout: wether to apply dropout (same dropout mask
            for every temporal slice of the input).
        input_dim: integer; optional dimensionality of the input.
        output_dim: integer; optional dimensionality of the output.
        timesteps: integer; optional number of timesteps.
        training: training phase tensor or boolean.
    # Returns
        Output tensor.
    """
    if not input_dim:
        input_dim = K.shape(x)[2]
    if not timesteps:
        timesteps = K.shape(x)[1]
    if not output_dim:
        output_dim = K.shape(w)[1]

    if dropout is not None and 0. < dropout < 1.:
        # apply the same dropout pattern at every timestep
        ones = K.ones_like(K.reshape(x[:, 0, :], (-1, input_dim)))
        dropout_matrix = K.dropout(ones, dropout)
        expanded_dropout_matrix = K.repeat(dropout_matrix, timesteps)
        x = K.in_train_phase(x * expanded_dropout_matrix, x, training=training)

    # collapse time dimension and batch dimension together
    x = K.reshape(x, (-1, input_dim))
    x = K.dot(x, w)
    if b is not None:
        x = K.bias_add(x, b)
    # reshape to 3D tensor
    if K.backend() == 'tensorflow':
        x = K.reshape(x, K.stack([-1, timesteps, output_dim]))
        x.set_shape([None, None, output_dim])
    else:
        x = K.reshape(x, (-1, timesteps, output_dim))
    return x

In [52]:
from keras.layers.recurrent import Recurrent
from keras import regularizers, constraints, initializers, activations
from keras.engine import InputSpec 

class AttentionDecoder(Recurrent):

    def __init__(self, units, output_dim,
                 activation='tanh',
                 return_probabilities=False,
                 name='AttentionDecoder',
                 kernel_initializer='glorot_uniform',
                 recurrent_initializer='orthogonal',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        """
        Implements an AttentionDecoder that takes in a sequence encoded by an
        encoder and outputs the decoded states 
        :param units: dimension of the hidden state and the attention matrices
        :param output_dim: the number of labels in the output space
        """
        self.units = units
        self.output_dim = output_dim
        self.return_probabilities = return_probabilities
        self.activation = activations.get(activation)
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.recurrent_initializer = initializers.get(recurrent_initializer)
        self.bias_initializer = initializers.get(bias_initializer)

        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.recurrent_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)

        self.kernel_constraint = constraints.get(kernel_constraint)
        self.recurrent_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)

        super(AttentionDecoder, self).__init__(**kwargs)
        self.name = name
        self.return_sequences = True  # must return sequences

    def build(self, input_shape):
        """
          See Appendix 2 of Bahdanau 2014, arXiv:1409.0473
          for model details that correspond to the matrices here.
        """

        self.batch_size, self.timesteps, self.input_dim = input_shape

        if self.stateful:
            super(AttentionDecoder, self).reset_states()

        self.states = [None, None]  # y, s

        """
            Matrices for creating the context vector
        """

        self.V_a = self.add_weight(shape=(self.units,),
                                   name='V_a',
                                   initializer=self.kernel_initializer,
                                   regularizer=self.kernel_regularizer,
                                   constraint=self.kernel_constraint)
        self.W_a = self.add_weight(shape=(self.units, self.units),
                                   name='W_a',
                                   initializer=self.kernel_initializer,
                                   regularizer=self.kernel_regularizer,
                                   constraint=self.kernel_constraint)
        self.U_a = self.add_weight(shape=(self.input_dim, self.units),
                                   name='U_a',
                                   initializer=self.kernel_initializer,
                                   regularizer=self.kernel_regularizer,
                                   constraint=self.kernel_constraint)
        self.b_a = self.add_weight(shape=(self.units,),
                                   name='b_a',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)
        """
            Matrices for the r (reset) gate
        """
        self.C_r = self.add_weight(shape=(self.input_dim, self.units),
                                   name='C_r',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_r = self.add_weight(shape=(self.units, self.units),
                                   name='U_r',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_r = self.add_weight(shape=(self.output_dim, self.units),
                                   name='W_r',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_r = self.add_weight(shape=(self.units, ),
                                   name='b_r',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)

        """
            Matrices for the z (update) gate
        """
        self.C_z = self.add_weight(shape=(self.input_dim, self.units),
                                   name='C_z',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_z = self.add_weight(shape=(self.units, self.units),
                                   name='U_z',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_z = self.add_weight(shape=(self.output_dim, self.units),
                                   name='W_z',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_z = self.add_weight(shape=(self.units, ),
                                   name='b_z',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)
        """
            Matrices for the proposal
        """
        self.C_p = self.add_weight(shape=(self.input_dim, self.units),
                                   name='C_p',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_p = self.add_weight(shape=(self.units, self.units),
                                   name='U_p',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_p = self.add_weight(shape=(self.output_dim, self.units),
                                   name='W_p',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_p = self.add_weight(shape=(self.units, ),
                                   name='b_p',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)
        """
            Matrices for making the final prediction vector
        """
        self.C_o = self.add_weight(shape=(self.input_dim, self.output_dim),
                                   name='C_o',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_o = self.add_weight(shape=(self.units, self.output_dim),
                                   name='U_o',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_o = self.add_weight(shape=(self.output_dim, self.output_dim),
                                   name='W_o',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_o = self.add_weight(shape=(self.output_dim, ),
                                   name='b_o',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)

        # For creating the initial state:
        self.W_s = self.add_weight(shape=(self.input_dim, self.units),
                                   name='W_s',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)

        self.input_spec = [
            InputSpec(shape=(self.batch_size, self.timesteps, self.input_dim))]
        self.built = True

    def call(self, x):
        # store the whole sequence so we can "attend" to it at each timestep
        self.x_seq = x

        # apply the a dense layer over the time dimension of the sequence
        # do it here because it doesn't depend on any previous steps
        # thefore we can save computation time:
        self._uxpb = _time_distributed_dense(self.x_seq, self.U_a, b=self.b_a,
                                             input_dim=self.input_dim,
                                             timesteps=self.timesteps,
                                             output_dim=self.units)

        return super(AttentionDecoder, self).call(x)

    def get_initial_state(self, inputs):
        print('inputs shape:', inputs.get_shape())

        # apply the matrix on the first time step to get the initial s0.
        s0 = activations.tanh(K.dot(inputs[:, 0], self.W_s))

        # from keras.layers.recurrent to initialize a vector of (batchsize,
        # output_dim)
        y0 = K.zeros_like(inputs)  # (samples, timesteps, input_dims)
        y0 = K.sum(y0, axis=(1, 2))  # (samples, )
        y0 = K.expand_dims(y0)  # (samples, 1)
        y0 = K.tile(y0, [1, self.output_dim])

        return [y0, s0]

    def step(self, x, states):

        ytm, stm = states

        # repeat the hidden state to the length of the sequence
        _stm = K.repeat(stm, self.timesteps)

        # now multiplty the weight matrix with the repeated hidden state
        _Wxstm = K.dot(_stm, self.W_a)

        # calculate the attention probabilities
        # this relates how much other timesteps contributed to this one.
        et = K.dot(activations.tanh(_Wxstm + self._uxpb),
                   K.expand_dims(self.V_a))
        at = K.exp(et)
        at_sum = K.sum(at, axis=1)
        at_sum_repeated = K.repeat(at_sum, self.timesteps)
        at /= at_sum_repeated  # vector of size (batchsize, timesteps, 1)

        # calculate the context vector
        context = K.squeeze(K.batch_dot(at, self.x_seq, axes=1), axis=1)
        # ~~~> calculate new hidden state
        # first calculate the "r" gate:

        rt = activations.sigmoid(
            K.dot(ytm, self.W_r)
            + K.dot(stm, self.U_r)
            + K.dot(context, self.C_r)
            + self.b_r)

        # now calculate the "z" gate
        zt = activations.sigmoid(
            K.dot(ytm, self.W_z)
            + K.dot(stm, self.U_z)
            + K.dot(context, self.C_z)
            + self.b_z)

        # calculate the proposal hidden state:
        s_tp = activations.tanh(
            K.dot(ytm, self.W_p)
            + K.dot((rt * stm), self.U_p)
            + K.dot(context, self.C_p)
            + self.b_p)

        # new hidden state:
        st = (1-zt)*stm + zt * s_tp

        yt = activations.softmax(
            K.dot(ytm, self.W_o)
            + K.dot(stm, self.U_o)
            + K.dot(context, self.C_o)
            + self.b_o)

        if self.return_probabilities:
            return at, [yt, st]
        else:
            return yt, [yt, st]

    def compute_output_shape(self, input_shape):
        """
            For Keras internal compatability checking
        """
        if self.return_probabilities:
            return (None, self.timesteps, self.timesteps)
        else:
            return (None, self.timesteps, self.output_dim)

    def get_config(self):
        """
            For rebuilding models on load time.
        """
        config = {
            'output_dim': self.output_dim,
            'units': self.units,
            'return_probabilities': self.return_probabilities
        }
        base_config = super(AttentionDecoder, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [71]:
from keras.layers import Flatten

def create_rnn_model(rnn, lstm, maxlen, embedding, max_features, embed_size,
                     rnn_dim=64, dense1_dim=100, dense2_dim=50,
                     embed_trainable=False, seed=123):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding],
                  trainable=embed_trainable)(inp)
    x = Dense(dense1_dim, activation='relu',
              kernel_initializer=glorot_uniform(seed=seed))(x)
    x = Bidirectional(rnn(rnn_dim, return_sequences=True,
                          kernel_initializer=glorot_uniform(seed=seed)))(x)
    #x = Attention(maxlen)(x)
    x = Bidirectional(lstm(rnn_dim, kernel_initializer=glorot_uniform(seed=seed),
                           return_sequences=True))(x)
    x = AttentionDecoder(rnn_dim,
                             name='attention_decoder_1',
                             output_dim=1,
                             return_probabilities=False,
                             trainable=True)(x) 
    ##x =  Flatten()
    ##x = Dense(1, activation='sigmoid')
    #x = GlobalMaxPooling1D()(x)
    #x = Dense(dense2_dim, activation='relu',
              #kernel_initializer=glorot_uniform(seed=seed))(x)
    #x = Dense(1, activation='sigmoid',
              #kernel_initializer=glorot_uniform(seed=seed))(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['acc'])
    return model

In [72]:
k = 8
num_val_samples = 200000 
num_epochs = 10
all_scores = []

In [69]:
temp = np.concatenate([train_y[:i * num_val_samples],
                train_y[(i+1) * num_val_samples:]],
                axis=0)

In [70]:
temp.shape

(800000,)

In [76]:
def f1_best(y_val, pred_val):
    best_f1 = 0
    best_thresh = 0
    for thresh in np.linspace(0.2, 0.4, 41):
        f1 = metrics.f1_score(y_val, (pred_val > thresh).astype(int))
        if f1 > best_f1:
            best_f1 = f1
            best_thresh = thresh
    return best_f1, best_thresh

embed_ids = [list(range(300)), list(range(300, 600)),
             list(range(600, 900)), list(range(900, 1200))]
embed_ids_dict = {1: [embed_ids[0], embed_ids[1], embed_ids[2], embed_ids[3]],
                  2: [embed_ids[0] + embed_ids[1],
                      embed_ids[0] + embed_ids[2],
                      embed_ids[0] + embed_ids[3],
                      embed_ids[1] + embed_ids[2],
                      embed_ids[1] + embed_ids[3],
                      embed_ids[2] + embed_ids[3]],
                  3: [embed_ids[0] + embed_ids[1] + embed_ids[2],
                      embed_ids[0] + embed_ids[1] + embed_ids[3],
                      embed_ids[0] + embed_ids[2] + embed_ids[3],
                      embed_ids[1] + embed_ids[2] + embed_ids[3]],
                  4: [embed_ids[0] + embed_ids[1] + embed_ids[2] + embed_ids[3]]}
embed_ids_lst = embed_ids_dict[2]
embed_size = 600

rnn = CuDNNGRU
lstm = CuDNNLSTM
embed_trainable = False

n_models = 6
#epochs = 7
batch_size = 512
dense1_dim = rnn_dim = 128
dense2_dim = 2 * rnn_dim

ema_n = int(len(train_y) / batch_size / 10)
decay = 0.9
scores = []

#pred_avg = np.zeros((len(val_y), 1))
#pred_test_avg = np.zeros((test_df.shape[0], 1))
for i in range(n_models):
    t1 = time.time()
    seed = 101 + 11 * i
    cols_in_use = embed_ids_lst[i % len(embed_ids_lst)]
    model = create_rnn_model(rnn, lstm, maxlen, embedding_matrix[:, cols_in_use],
                             max_features, embed_size,
                             rnn_dim=rnn_dim,
                             dense1_dim=dense1_dim,
                             dense2_dim=dense2_dim,
                             embed_trainable=embed_trainable,
                             seed=seed)
    model.summary()
    ema = ExponentialMovingAverage(model, decay=decay, mode='batch', n=ema_n)
    
    #cross_validation 
    for i in range(k):
        print(f'Processing fold # {i}')
        val_data = train_X[i * num_val_samples: (i+1) * num_val_samples]
        val_targets = train_y[i * num_val_samples: (i+1) * num_val_samples]
    
        partial_train_data = np.concatenate(
                                [train_X[:i * num_val_samples],
                                train_X[(i+1) * num_val_samples:]],
                                axis=0)
        partial_train_targets = np.concatenate(
                                [train_y[:i * num_val_samples],
                                train_y[(i+1)*num_val_samples:]],
                                axis=0)
        res = model.fit(x=partial_train_data,
                        y=partial_train_targets,
                        batch_size=batch_size,
                        epochs = num_epochs,
                        verbose=1,
                        validation_data = (val_data,val_targets))
        print("Scores: ", res)
    
    #model.fit(train_X, train_y, batch_size=batch_size, epochs=epochs,
              #callbacks=[ema], verbose=0)
    m = ema.ema_model
    t_per_epoch = (time.time() - t1) / epochs
    pred = m.predict([val_X])
    pred_avg += pred
    pred_test = m.predict([test_X])
    pred_test_avg += pred_test
    f1_one, thresh_one = f1_best(val_y, pred)
    f1_avg, thresh_avg = f1_best(val_y, pred_avg / (i + 1))
    nll_one = metrics.log_loss(val_y, pred)
    nll_avg = metrics.log_loss(val_y, pred_avg / (i + 1))
    auc_one = metrics.roc_auc_score(val_y, pred)
    auc_avg = metrics.roc_auc_score(val_y, pred_avg)
    print(f'  n_model:{i + 1} epoch:{epochs} F1:{f1_avg:.4f} th:{thresh_avg:.3f} ' +
          f'AUC:{auc_avg:.4f} NLL:{nll_avg:.4f} One:{f1_one:.4f} {auc_one:.4f} {nll_one:.4f} ' +
          f'Time:{time.time() - t1:.1f}s  {t_per_epoch:.1f}s/epoch')

inputs shape: (?, 50, 256)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 50)                0         
_________________________________________________________________
embedding_12 (Embedding)     (None, 50, 600)           54000000  
_________________________________________________________________
dense_15 (Dense)             (None, 50, 128)           76928     
_________________________________________________________________
bidirectional_23 (Bidirectio (None, 50, 256)           198144    
_________________________________________________________________
bidirectional_24 (Bidirectio (None, 50, 256)           395264    
_________________________________________________________________
attention_decoder_1 (Attenti (None, 50, 1)             230786    
Total params: 54,901,122
Trainable params: 901,122
Non-trainable params: 54,000,000
_______________________________

ValueError: Error when checking target: expected attention_decoder_1 to have 3 dimensions, but got array with shape (800000, 1)

In [ ]:
pred_test_avg /= n_models
pred_test_avg = (pred_test_avg>thresh_avg).astype(int)
out_df = pd.DataFrame({"qid":test_df["qid"].values})
out_df['prediction'] = pred_test_avg
out_df.to_csv("submission.csv", index=False)

print(f'Done:{time.time() - t0:.1f}s')

In [ ]:
print(out_df)